In [1]:
%load_ext autoreload
%autoreload 2
%aimport -decode_fish.engine.place_psfs

In [2]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = '1'

from decode_fish.imports import *
from decode_fish.funcs.file_io import *
from decode_fish.funcs.emitter_io import *
from decode_fish.funcs.utils import *
from decode_fish.funcs.dataset import *
from decode_fish.funcs.plotting import *
from decode_fish.engine.noise import estimate_noise_scale
from decode_fish.engine.microscope import add_pos_noise
import shutil
from decode_fish.funcs.visualization import *
from decode_fish.funcs.evaluation import *
from decode_fish.funcs.predict import window_predict
from decode_fish.funcs.evaluation import matching
from decode_fish.funcs.merfish_eval import *
from decode_fish.funcs.exp_specific import *
from decode_fish.funcs.routines import *

from omegaconf import open_dict
from hydra import compose, initialize

In [3]:
# import os
# os.environ["CUDA_VISIBLE_DEVICES"] = '1'

In [8]:
initialize(config_path="../config")

hydra.initialize()

In [168]:
cfg = compose(config_name='train.yaml')
cfg = OmegaConf.create(cfg)

## Simple simulations for comparison with ISTDECO (no z, constant BG)

In [134]:
# project and run_name set the directories where the trained models are saved, as well as the structure in wandb.ai
cfg.output.project = 'MERFISH_sim_2D'
cfg.run_name ='sim_2d_1'

cfg.codebook._target_ = 'decode_fish.funcs.exp_specific.get_starfish_codebook'

cfg.genm.exp_type.n_channels = 16
cfg.genm.exp_type.n_genes = 140
cfg.genm.exp_type.n_bits = 4
cfg.genm.exp_type.sample_from_codebook = True
cfg.genm.exp_type.pred_z = False

cfg.data_path.image_sim.image_shape = [1,16,1,48,48]

'''Simulation parameters'''
cfg.genm.PSF.gauss_radii = [1.,1.5,1.5]

cfg.genm.foci.n_foci_avg = 0
    
cfg.genm.prob_generator.low = 0.0002
cfg.genm.prob_generator.high = 0.1

cfg.genm.intensity_dist.int_conc = 4.
cfg.genm.intensity_dist.int_rate = 1.
cfg.genm.intensity_dist.int_loc = .5

cfg.genm.microscope.scale = 100
cfg.genm.emitter_noise.rate_fac = 0.

cfg.genm.pos_noise.pos_noise_z = 0.
cfg.genm.pos_noise.pos_noise_xy = 0.
cfg.training.z_sig_fac = 1.


'''Performance tracking'''
# Set to 'disabled' if you do not have a wandb account.
cfg.output.wandb_mode = 'online'
cfg.output.log_interval = 500

'''Network architecture'''
cfg.network.f_maps = 256
cfg.network.depth = 2
cfg.network.is_2D = True

cfg.network.inp_scale = 300
cfg.network.inp_offset = 100 
'''Training'''
cfg.training.bs = 20
cfg.training.net.opt.lr = 5e-4
cfg.training.mic.norm_reg = 0.
cfg.training.net.bl_loss_scale = 0.0

cfg.training.num_iters = 30000

cfg.training.int.enabled = False
cfg.training.mic.enabled = False

cfg.data_path.model_init = None

Save cfg

In [135]:
fname = f'../config/experiment/{cfg.run_name}.yaml'
OmegaConf.save(cfg, fname)
prepend_line(fname, '# @package _global_')
cfg.run_name

'sim_2d_1'

In [110]:
home_dir = os.getenv("HOME")
print(f'bsub -gpu "num=1" -q gpu_rtx -o {home_dir}/logs/ofish.log -e {home_dir}/logs/efish.log {home_dir}/anaconda3/envs/decode_fish_dev2/bin/python {home_dir}/Mackebox/Artur/WorkDB/deepstorm/decode_fish/decode_fish/train.py +experiment={cfg.run_name} run_name=norm:none output.group={cfg.run_name}')

bsub -gpu "num=1" -q gpu_rtx -o /groups/turaga/home/speisera/logs/ofish.log -e /groups/turaga/home/speisera/logs/efish.log /groups/turaga/home/speisera/anaconda3/envs/decode_fish_dev2/bin/python /groups/turaga/home/speisera/Mackebox/Artur/WorkDB/deepstorm/decode_fish/decode_fish/train.py +experiment=sim_2d_1 run_name=norm:none output.group=sim_2d_1


In [104]:
img_3d, decode_dl = get_dataloader(cfg)
psf, noise, micro = load_psf_noise_micro(cfg)

Crop size larger than volume in at least one dimension. Crop size changed to (1, 48, 48)
1 volumes


## Simulations for internal testing of gen. model learning

In [136]:
## Simple simulations for comparison with ISTDECO (no z, constant BG)# get_simulation_statistics(decode_dl, micro, **cfg.genm.intensity_dist, int_threshold=0.1, samples=5, channels=cfg.genm.exp_type.n_channels, n_bits=cfg.genm.exp_type.n_bits, psf_noise=False)

In [492]:
cfg = compose(config_name='train.yaml', overrides= ["+exp_type=merfish_MOp"])
del(cfg.data_path.image_proc.override)

In [493]:
thetas = 1.
channel_facs = [1.] * 22

In [500]:
cfg.output.project = 'MERFISH_sim_int'
cfg.run_name ='sim_int_6_os'

cfg.data_path.image_path = '/groups/turaga/home/speisera/Mackebox/Artur/WorkDB/deepstorm/decode_fish/data/merfish_sims/sim_int_6_os/images.tiff'

cfg.genm.foci.n_foci_avg = 0

cfg.genm.prob_generator.low = 0.02
cfg.genm.prob_generator.high = 0.06

cfg.output.wandb_mode = 'online'

cfg.network.f_maps = 384
cfg.network.depth = 2
cfg.network.n_out_l = 1
cfg.training.bs = 10

cfg.genm.noise.theta = thetas
cfg.genm.microscope.ch_facs = channel_facs
cfg.genm.microscope.col_shifts_enabled = False

cfg.sim.roi_mask.percentile = 20
cfg.sim.random_crop.crop_sz = 56

cfg.sim.bg_estimation.smoothing.smoothing_filter_size = 9

cfg.sim.bg_estimation.type = 'smoothing'
# cfg.sim.bg_estimation.uniform.img_mean = True
# cfg.sim.bg_estimation.type = 'uniform'

cfg.genm.intensity_dist.int_conc = 5.
cfg.genm.intensity_dist.int_rate = 1.
cfg.genm.intensity_dist.int_loc = 1.

cfg.training.num_iters = 35000
cfg.training.mic.enabled = False

cfg.genm.microscope.scale = 10000
cfg.genm.phasing = 0.0

cfg.genm.PSF.n_cols = 1
cfg.genm.microscope.ch_cols = [0] * 22

cfg.data_path.psf_path = '/groups/turaga/home/speisera/Mackebox/Artur/WorkDB/deepstorm/decode_fish/figures/MF_psf_07_20n.tif'
cfg.genm.PSF.gauss_radii = [1.,1.,1.1]
cfg.genm.PSF.psf_extent_zyx = [7,21,21]
# cfg.data_path.psf_path = None
cfg.genm.microscope.norm = 'escort'
cfg.genm.PSF.mode = 'bicubic'

cfg.training.mic.mean_diff = True

cfg.training.mic.par_grads.channel_facs = True
cfg.training.mic.par_grads.channel_shifts = True
cfg.training.mic.par_grads.theta_par = False
cfg.training.mic.par_grads.psf_vol = True
cfg.training.mic.par_grads.color_shifts = False
cfg.training.mic.par_grads.z_facs = False
cfg.training.mic.par_grads.sc_fac = False

cfg.training.net.enabled = True

cfg.training.start_mic = 10000
cfg.training.start_int = 100000

cfg.output.log_interval = 600

cfg.genm.emitter_noise.rate_fac = 3.

cfg.training.net.opt.lr = 3e-4
cfg.training.z_sig_fac = 1.

cfg.evaluation.code_stats.enabled = False
cfg.training.schedule = None
cfg.training.schedule = [10000, 2000, 5000, 1000, 5000, 1000, 5000, 1000, 10000]
cfg.network.chrom_map = False

cfg.training.mic.opt.lr = 0.001

cfg.training.psf.opt.lr = 0.00005
cfg.training.psf.sched.step_size = 2000
cfg.training.psf.l1_reg = 0.0003

cfg.training.net.bl_loss_scale = .003

cfg.data_path.model_init  = None
cfg.data_path.micro_init  = None

# cfg.data_path.micro_init  = '/groups/turaga/home/speisera/Mackebox/Artur/WorkDB/deepstorm/decode_fish/data/merfish_sims/sim_int_5_osc/'
# cfg.data_path.model_init  = '/groups/turaga/home/speisera/Mackebox/Artur/WorkDB/deepstorm//models/fishcod/MERFISH_sim_int/sim_int_5_osc/ae_train_mae//checkpoint/'
cfg.genm.exp_type.const_theta_sim = True

cfg.network.is_2D = True
cfg.network.pred_z = True

cfg.sim.bg_estimation.shuffle_ch = True
cfg.training.pred_em_noise = False
cfg.genm.exp_type.em_noise_inf = True

In [501]:
cfg.genm.exp_type.const_theta_sim

True

In [502]:
fname = f'../config/experiment/{cfg.run_name}.yaml'
OmegaConf.save(cfg, fname)
prepend_line(fname, '# @package _global_')
cfg.run_name

'sim_int_6_os'

In [503]:
name = 'sl_true_gen'
home_dir = os.getenv("HOME")
print(f'bsub -gpu "num=1" -q gpu_rtx -o {home_dir}/logs/ofish.log -e {home_dir}/logs/efish.log {home_dir}/anaconda3/envs/decode_fish_dev2/bin/python {home_dir}/Mackebox/Artur/WorkDB/deepstorm/decode_fish/decode_fish/train.py +experiment={cfg.run_name} training.schedule=Null run_name={name} output.group={cfg.run_name} training.num_iters=35000 data_path.psf_path=Null data_path.model_init=null data_path.micro_init=/groups/turaga/home/speisera/Mackebox/Artur/WorkDB/deepstorm/decode_fish/data/merfish_sims/sim_int_6_os/')

bsub -gpu "num=1" -q gpu_rtx -o /groups/turaga/home/speisera/logs/ofish.log -e /groups/turaga/home/speisera/logs/efish.log /groups/turaga/home/speisera/anaconda3/envs/decode_fish_dev2/bin/python /groups/turaga/home/speisera/Mackebox/Artur/WorkDB/deepstorm/decode_fish/decode_fish/train.py +experiment=sim_int_6_os training.schedule=Null run_name=sl_true_gen output.group=sim_int_6_os training.num_iters=35000 data_path.psf_path=Null data_path.model_init=null data_path.micro_init=/groups/turaga/home/speisera/Mackebox/Artur/WorkDB/deepstorm/decode_fish/data/merfish_sims/sim_int_6_os/


In [504]:
name = 'ae_train'
home_dir = os.getenv("HOME")
print(f'bsub -gpu "num=1" -q gpu_rtx -o {home_dir}/logs/ofish.log -e {home_dir}/logs/efish.log {home_dir}/anaconda3/envs/decode_fish_dev2/bin/python {home_dir}/Mackebox/Artur/WorkDB/deepstorm/decode_fish/decode_fish/train.py +experiment={cfg.run_name} run_name={name} output.group={cfg.run_name}')

bsub -gpu "num=1" -q gpu_rtx -o /groups/turaga/home/speisera/logs/ofish.log -e /groups/turaga/home/speisera/logs/efish.log /groups/turaga/home/speisera/anaconda3/envs/decode_fish_dev2/bin/python /groups/turaga/home/speisera/Mackebox/Artur/WorkDB/deepstorm/decode_fish/decode_fish/train.py +experiment=sim_int_6_os run_name=ae_train output.group=sim_int_6_os


In [505]:
name = 'sl_no_learn'
home_dir = os.getenv("HOME")
print(f'bsub -gpu "num=1" -q gpu_rtx -o {home_dir}/logs/ofish.log -e {home_dir}/logs/efish.log {home_dir}/anaconda3/envs/decode_fish_dev2/bin/python {home_dir}/Mackebox/Artur/WorkDB/deepstorm/decode_fish/decode_fish/train.py +experiment={cfg.run_name} run_name={name} output.group={cfg.run_name} training.schedule=Null')

bsub -gpu "num=1" -q gpu_rtx -o /groups/turaga/home/speisera/logs/ofish.log -e /groups/turaga/home/speisera/logs/efish.log /groups/turaga/home/speisera/anaconda3/envs/decode_fish_dev2/bin/python /groups/turaga/home/speisera/Mackebox/Artur/WorkDB/deepstorm/decode_fish/decode_fish/train.py +experiment=sim_int_6_os run_name=sl_no_learn output.group=sim_int_6_os training.schedule=Null


In [33]:
# get_simulation_statistics(decode_dl, micro, **cfg.genm.intensity_dist, int_threshold=0.1, samples=10, channels=cfg.genm.exp_type.n_channels, n_bits=cfg.genm.exp_type.n_bits, psf_noise=False)

In [285]:
os.makedirs('tra', exist_ok=True)

In [349]:
cfg.training.resume

False